In [1]:
from dataset import BPI2012Dataset
from parameters.dataset import BPI2012Parameters
from utils.save import get_json_dict
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from parameters.model import LSTMPredNextWithResourceModelParameters
from model import LSTMPredNextWithResourceModel
from controller import TrainingController

In [2]:
bpi_2012_parms = BPI2012Parameters()

In [3]:
get_json_dict(bpi_2012_parms)

{'file_path': './data/event_logs/BPI_Challenge_2012.xes',
 'preprocessed_folder_path': './data/preprocessed/BPI_Challenge_2012_with_resource',
 'include_types': ['A', 'O', 'W'],
 'include_complete_only': True}

In [4]:
dataset = BPI2012Dataset(bpi_2012_parms)


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [5]:
model_params = LSTMPredNextWithResourceModelParameters()

In [6]:
get_json_dict(model_params)

{'activity_embedding_dim': 32,
 'resource_embedding_dim': 128,
 'lstm_hidden': 64,
 'dense_dim': 64,
 'dropout': 0.1}

In [7]:
model = LSTMPredNextWithResourceModel(
    activity_vocab = dataset.activity_vocab,
    resource_vocab = dataset.resource_vocab,
    parameters = model_params
)

In [8]:
train_params = TrainingParameters(
    stop_epoch=20,
)

In [9]:
get_json_dict(train_params)

{'stop_epoch': 20,
 'batch_size': 128,
 'verbose_freq': 250,
 'run_validation_freq': 80,
 'train_test_split_portion': [0.8, 0.1],
 'random_seed': 12345}

In [10]:
loss_params = LossParameters()

In [11]:
get_json_dict(loss_params)

{'loss': 'CrossEntropy'}

In [12]:
optim_params = OptimizerParameters()

In [13]:
get_json_dict(optim_params)

{'optimizer': 'Adam',
 'learning_rate': 0.005,
 'l2': 1e-10,
 'lr_scheduler': 'ExponentialDecay',
 'lr_step_scheduler_step': 800,
 'lr_setp_scheduler_gamma': 0.8,
 'lr_exp_decay_scheduler_step': 1000,
 'lr_exp_decay_scheduler_rate': 0.96,
 'lr_exp_decay_scheduler_staircase': True,
 'SGD_momentum': 0.9}

In [14]:
controller = TrainingController(
    dataset,
    model,
    train_params,
    optim_params,
    loss_params,
)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  


In [16]:
controller.model_info()

Model: "LSTMPredNextWithResourceModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  49408     
_________________________________________________________________
sequential (Sequential)      (1, 1, 26)                10782     
Total params: 95,070
Trainable params: 94,684
Non-trainable params: 386
_________________________________________________________________


In [17]:
# tensorboard --logdir logs/gradient_tape --host localhost --port 8088
test_accuracy = controller.train()


| Training records in logs/gradient_tape/20210629-033522 

| Total epochs: 10 

| Total steps: 820 

| Start epoch 0 

| Evaluation result | Loss [0.7592] | Accuracy [0.0730]  

| Start epoch 1 

| Evaluation result | Loss [0.8625] | Accuracy [0.0730]  

| Start epoch 2 

| Evaluation result | Loss [0.7147] | Accuracy [0.0855]  

| Start epoch 3 

| Evaluation result | Loss [0.4342] | Accuracy [0.3257]  

| Start epoch 4 

| Evaluation result | Loss [0.2262] | Accuracy [0.7118]  

| Start epoch 5 

| Evaluation result | Loss [0.1486] | Accuracy [0.8175]  

| Start epoch 6 

| Evaluation result | Loss [0.1263] | Accuracy [0.8242]  

| Start epoch 7 

| Evaluation result | Loss [0.1219] | Accuracy [0.8230]  

| Start epoch 8 

| Evaluation result | Loss [0.1221] | Accuracy [0.8217]  

| Start epoch 9 

| Evaluation result | Loss [0.1232] | Accuracy [0.8222]  

| Performing Testing ... 

| Evaluation result | Loss [0.1163] | Accuracy [0.8199]  


In [ ]:
# test_accuracy = controller.perform_eval_on_testset()

In [18]:
save_folder_path = model.get_folder_path("bpi_2012_pred_next_traning.ipynb", test_accuracy , dataset.info_str())

In [19]:
controller.save_parameters(save_folder_path)
model.save(save_folder_path)


| Parameters saved successfully to: ./SavedModels/0.8199_LSTMPredNextWithResourceModel_AOW_CompleteOnly_2021-06-29 03:44:59.619813  

| Vocabs saved successfully to: ./SavedModels/0.8199_LSTMPredNextWithResourceModel_AOW_CompleteOnly_2021-06-29 03:44:59.619813  

| Model saved successfully to: ./SavedModels/0.8199_LSTMPredNextWithResourceModel_AOW_CompleteOnly_2021-06-29 03:44:59.619813  


In [ ]:
# controller.load_parameters(savet_folder_path)
# model.load(save_folder_path)